# Test embedalign with SentEval 

This notebook will allow you to test EmbedAlign using SentEval. In particular, this also works on **CPUs** :D

* Dependencies:
    * Python 3.5 with NumPy/SciPy
    * Pytorch 
    * Tensorflow 1.5.0  (for CPUs or GPUs depending on how you plan to run it)
        * For example in MacOS: 
        ```
        pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.5.0-py3-none-any.whl
        ```
    * scikit-learn>=0.18.0
    * dill>=0.2.7.1


* Install `dgm4nlp` by following the instructions [here](https://github.com/uva-slpl/dgm4nlp), we highly recommend the use of `virtualenv`.

In the same `virtualenv`, do the following:

* Clone repo from FAIR github
```
    git clone https://github.com/facebookresearch/SentEval.git
    cd SentEval/
```

* Install senteval
```
    python setup.py install
```

* Download datasets (it takes some time...)
    * these are downstream tasks
    * new Senteval also has probing tasks (https://github.com/facebookresearch/SentEval/tree/master/data/probing) for evaluating linguistic properties of your embeddings. 

```
    cd data/downstream/
    ./get_transfer_data.bash
```

* Download [pretained embedlaign model](https://surfdrive.surf.nl/files/index.php/s/9M4h5zqmYETSmf3)


* The following code evaluates embedalign pretrained embeddings on en-fr Europarl on different NLP downstream tasks.



In [1]:
from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
#import data 
# data.py is part of Senteval and it is used for loading word2vec style files
import senteval
import tensorflow as tf
import logging
from collections import defaultdict
import dill
import dgm4nlp

In [2]:
class dotdict(dict):
    """ dot.notation access to dictionary attributes """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

class EmbeddingExtractor:
    """
    This will compute a forward pass with the inference model of EmbedAlign and 
        give you the variational mean for each L1 word in the batch.
        
    Note that this takes monolingual L1 sentences only (at this point we have a traiend EmbedAlign model
        which dispenses with L2 sentences).    
        
    You don't really want to touch anything in this class.
    """

    def __init__(self, graph_file, ckpt_path, config=None):        
        g1 = tf.Graph()
        self.meta_graph = graph_file
        self.ckpt_path = ckpt_path
        
        self.softmax_approximation = 'botev-batch' #default
        with g1.as_default():
            self.sess = tf.Session(config=config, graph=g1)
            # load architecture computational graph
            self.new_saver = tf.train.import_meta_graph(self.meta_graph)
            # restore checkpoint
            self.new_saver.restore(self.sess, self.ckpt_path) #tf.train.latest_checkpoint(
            self.graph = g1  #tf.get_default_graph()
            # retrieve input variable
            self.x = self.graph.get_tensor_by_name("X:0")
            # retrieve training switch variable (True:trianing, False:Test)
            self.training_phase = self.graph.get_tensor_by_name("training_phase:0")
            #self.keep_prob = self.graph.get_tensor_by_name("keep_prob:0")

    def get_z_embedding_batch(self, x_batch):
        """
        :param x_batch: is np array of shape [batch_size, longest_sentence] containing the unique ids of words
        
        :returns: [batch_size, longest_sentence, z_dim]        
        """
        # Retrieve embeddings from latent variable Z
        # we can sempale several n_samples, default 1
        try:
            z_mean = self.graph.get_tensor_by_name("z:0")
            
            feed_dict = {
                self.x: x_batch,
                self.training_phase: False,
                #self.keep_prob: 1.

            }
            z_rep_values = self.sess.run(z_mean, feed_dict=feed_dict) 
        except:
            raise ValueError('tensor Z not in graph!')
        return z_rep_values

This is how you interface with SentEval. The only think you need to change are the paths to trained models in the main block at the end.

In [11]:
# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': '',
                   'usepytorch': False,
                   'kfold': 10,
                   'ckpt_path': '',
                   'tok_path': '',
                   'extractor': None,
                   'tks1': None}
# made dictionary a dotdict
params_senteval = dotdict(params_senteval)
# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}



def prepare(params, samples):
    """
    In this example we are going to load a tensorflow model, 
    we open a dictionary with the indices of tokens and the computation graph
    """
    params.extractor = EmbeddingExtractor(
        graph_file='%s.meta'%(params.ckpt_path),
        ckpt_path=params.ckpt_path,
        config=None #run in cpu
    )

    # load tokenizer from training
    params.tks1 = dill.load(open(params.tok_path, 'rb'))
    return

def batcher(params, batch):
    """
    At this point batch is a python list containing sentences. Each sentence is a list of tokens (each token a string).
    The code below will take care of converting this to unique ids that EmbedAlign can understand.
    
    This function should return a single vector representation per sentence in the batch.
    In this example we use the average of word embeddings (as predicted by EmbedAlign) as a sentence representation.
    
    In this method you can do mini-batching or you can process sentences 1 at a time (batches of size 1).
    We choose to do it 1 sentence at a time to avoid having to deal with masking. 
    
    This should not be too slow, and it also saves memory.
    """
    
    # if a sentence is empty dot is set to be the only token
    # you can change it into NULL dependening in your model
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []
    for sent in batch:
        # Here is where dgm4nlp converts strings to unique ids respecting the vocabulary
        # of the pre-trained EmbedAlign model
        # from tokens ot ids position 0 is en
        x1 = params.tks1[0].to_sequences([(' '.join(sent))])
        
        # extract word embeddings in context for a sentence
        # [1, sentence_length, z_dim]
        z_batch1 = params.extractor.get_z_embedding_batch(x_batch=x1)
        # sentence vector is the mean of word embeddings in context
        # [1, z_dim]
        sent_vec = np.mean(z_batch1, axis=1)
        # check if there is any NaN in vector (they appear sometimes when there's padding)
        if np.isnan(sent_vec.sum()):
            sent_vec = np.nan_to_num(sent_vec)        
        embeddings.append(sent_vec)
    if len(embeddings) == 0:
        return None
    embeddings = np.vstack(embeddings)
    return embeddings


# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)


# define paths
# path to senteval data
# note senteval adds downstream into the path
params_senteval.task_path = 'SentEval/data/' 
# path to computation graph
# we use best model on validation AER
# TODO: you have to point to valid paths! Use the pre-trained model linked from the top of this notebook.
params_senteval.ckpt_path = 'models/ull-practical3-embedalign/model.best.validation.aer.ckpt'
# path to tokenizer with ids of trained Europarl data
# out dictionary id depends on dill for pickle
params_senteval.tok_path = 'models/ull-practical3-embedalign/tokenizer.pickle'
# we use 10 fold cross validation
params_senteval.kfold = 10
se = senteval.engine.SE(params_senteval, batcher, prepare)

# here you define the NLP taks that your embedding model is going to be evaluated
# in (https://arxiv.org/abs/1802.05883) we use the following :
# SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
# but STS14 (semantic textual similarity) is a similar type of semantic task
transfer_tasks = ["MR" ,"CR" ,"SUBJ" ,"MPQA" ,"SST" ,"TREC" ,"SICK-E" ,"SNLI" ,"MRPC" ,"STS 2012" ,"STS 2013" ,"STS 2014" ,"STS 2015" ,"STS 2016" ,"STS B" ,"SICK-R" ,"COCO"]
# senteval prints the results and returns a dictionary with the scores
results = se.eval(transfer_tasks)
print(results)

2018-05-21 14:51:48,146 : ***** Transfer task : MR *****




INFO:tensorflow:Restoring parameters from models/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-21 14:51:50,209 : Restoring parameters from models/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-21 14:51:50,852 : Generating sentence embeddings


ValueError: tensor Z not in graph!

In [8]:
import pickle as pkl

pkl.dump(results, open("embed_align_results.pkl", "wb"))

In [9]:
import os
import matplotlib.pyplot as plt
%matplotlib inline

2018-05-21 14:43:40,745 : $HOME=/home/misfit
2018-05-21 14:43:40,747 : matplotlib data path /home/misfit/.pyenv/versions/3.5.5/envs/env-ull/lib/python3.5/site-packages/matplotlib/mpl-data
2018-05-21 14:43:40,756 : loaded rc file /home/misfit/.pyenv/versions/3.5.5/envs/env-ull/lib/python3.5/site-packages/matplotlib/mpl-data/matplotlibrc
2018-05-21 14:43:40,759 : matplotlib version 2.2.2
2018-05-21 14:43:40,760 : interactive is False
2018-05-21 14:43:40,762 : platform is linux
2018-05-21 14:43:40,764 : loaded modules: ['tensorflow.python.ops.metrics', 'six.moves', 'tensorflow.python.ops.gen_ctc_ops', 'tensorflow.python.data.ops', 'tensorflow.python.profiler.option_builder', 'encodings.idna', '_bz2', 'concurrent.futures._base', 'sklearn.externals.joblib.disk', 'tensorflow.python.keras.layers', 'sklearn.svm.classes', 'IPython.terminal.embed', 'numpy.lib.arraysetops', 'torch.nn.modules.distance', 'tensorflow.python.ops.state_grad', 'scipy.stats._binned_statistic', 'tensorflow.core', 'pygmen

2018-05-21 14:43:40,873 : CACHEDIR=/home/misfit/.cache/matplotlib
2018-05-21 14:43:40,883 : Using fontManager instance from /home/misfit/.cache/matplotlib/fontList.json
2018-05-21 14:43:41,127 : backend module://ipykernel.pylab.backend_inline version unknown
2018-05-21 14:43:41,146 : backend module://ipykernel.pylab.backend_inline version unknown
2018-05-21 14:43:41,158 : backend module://ipykernel.pylab.backend_inline version unknown


In [10]:
plt.rcParams["figure.figsize"] = (15, 10)
plots_path = "./plots"

results.keys()

dict_keys(['SICKEntailment', 'CR', 'MR', 'SST2', 'MPQA', 'MRPC', 'SUBJ', 'TREC', 'STS14'])